In [1]:
# Loading the required libraries
import tarfile
import numpy as np
import pandas as pd

The dataset being used in this study is pertaining to data collected from edge servers of the Indiana University network. The servers collect the HTTP requests made on the network and report the data back to the holders that maintain the record in raw data files.<br><br>
The dataset was available in form of a .tgz file that needed to be read and pre-processed

In [2]:
# # Reading the tarfile
# tar = tarfile.open("C:/Users/vyom/Downloads/web-clicks-nov-2009/web-clicks-nov-2009.tar")
# # tar.getmembers()

# for member in tar.getmembers():
#     f=tar.extractfile(member)
#     content=f.read()

In [3]:
# Reading the file as csv
# This file is currently being loaded from local. Please find the gdrive location for this file in the README file
df = pd.read_csv('C:/Users/vyom/Downloads/web-clicks-nov-2009.tgz', compression='gzip', header=0, sep=' ', quotechar='"', error_bad_lines=False)

df.shape

b'Skipping line 1243989: expected 11 fields, saw 15\n'
b'Skipping line 1247438: expected 11 fields, saw 15\n'
b'Skipping line 15502608: expected 11 fields, saw 15\n'
b'Skipping line 20842016: expected 11 fields, saw 15\n'
C:\Users\vyom\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(22133644, 11)

In [4]:
# Checking the dataframe format
df.head()

,2009-11-01.json,0,Unnamed: 2,Unnamed: 3,"1,",timestamp:,"1257033601,",from:,"theybf.com,",to:,w.sharethis.com}
0,"{""count"":","1,",timestamp:,"1257033601,",from:,",",to:,agohq.org},NaN,NaN,NaN
1,"{""count"":","3,",timestamp:,"1257033601,",from:,"twistysdownload.com,",to:,adserving.cpxinteractive.com},NaN,NaN,NaN
2,"{""count"":","1,",timestamp:,"1257033601,",from:,"459.cim.meebo.com,",to:,459.cim.meebo.com},NaN,NaN,NaN
3,"{""count"":","1,",timestamp:,"1257033601,",from:,"boards.nbc.com,",to:,change.menelgame.pl},NaN,NaN,NaN
4,"{""count"":","1,",timestamp:,"1257033601,",from:,"mail3-12.sinamail.sina.com.cn,",to:,mail3-12.sinamail.sina.com.cn},NaN,NaN,NaN


Having loaded the data in form of a csv and stored in a dataframe, it is evidently not clean, and needs to be processed. There are 3 major columns of interest which will be picked from the above dataset:
- the referer website
- the destination website
- the count of users

In [5]:
# Renaming the columns and filtering out the required columns
df.columns = range(len(df.columns))
dataset = df[[5, 7, 1]]

In [6]:
# Renaming the columns
dataset.columns = ['from_url', 'to_url', 'user_count']

Having picked the relevant columns, we'll cache the data on the local storage

In [7]:
# Caching the dataframe in local, whcih needs to be cleaned further
dataset.to_csv('C:/Users/vyom/Downloads/web-clicks-cleaned.csv', header=True)

In [8]:
# Restarting the kernel, and reading the cached dataframe to proceed with the datacleaning
dataset = pd.read_csv('C:/Users/vyom/Downloads/web-clicks-cleaned.csv', index_col=0)

C:\Users\vyom\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Now that we have the relevant columns, there is plenty of pre-processing that needs to be done. The below sets of codes perform the following types of data cleaning to have the data ready for EDA and modelling:
- Some of the websites provided in the dataset are entire URLs of web pages i.e. they include the domain name along with the remaining section of the webpage. Therefore, all sections of the URL are eliminated apart from just the website domain name. This is done by removing all character after (and including) the character ‘/’.
- The data consisted of websites from across different countries and organizations. Only the domains with extensions relevant to the UK were considered i.e. domain.com, domain.net, domain.co.uk, domain.edu, domain.org, domain.gov
- Filtering out websites that contain punctuation characters that are not associated with the usual format of the websites e.g. ‘%’, ‘/’, ‘\’, ‘”’, ‘;’, ‘=’.
- Filtering out websites that hold explicit/unrestrained content.
- Filtering out all the combination of websites that have less than 50 users in common.
- Filtering out entries from the dataset that have the same referrer and destination websites.


In [9]:
# Cleaning the user_count column
dataset['user_count'] = dataset['user_count'].str.replace(',', '')
dataset = dataset.fillna(0)
dataset['user_count'] = dataset['user_count'].astype(int)

In [10]:
# Cleaning the url columns
dataset['from_url'] = dataset['from_url'].str.replace(',', '')
dataset['to_url'] = dataset['to_url'].str.replace('}', '')

In [11]:
# Filtering out rows with 0 counts - erronour values
dataset = dataset.loc[dataset['user_count']!=0, :]

In [12]:
# Filtering out for the most common url extensions
searchfor = ['.com', '.net', '.co.uk', '.edu', '.org', '.gov']
dataset = dataset.loc[dataset['from_url'].str.contains('|'.join(searchfor)), :]
dataset = dataset.loc[dataset['to_url'].str.contains('|'.join(searchfor)), :]

# Filtering out the url columns that contain data not associated with the format of websites
dataset = dataset.loc[dataset['from_url'].str.contains('.', regex=False),:]
dataset = dataset.loc[dataset['to_url'].str.contains('.', regex=False),:]

dataset = dataset.loc[~dataset['from_url'].str.contains('%'),:]
dataset = dataset.loc[~dataset['to_url'].str.contains('%'),:]

dataset = dataset.loc[~dataset['from_url'].str.contains('='),:]
dataset = dataset.loc[~dataset['to_url'].str.contains('='),:]

dataset = dataset.loc[~dataset['from_url'].str.contains('/'),:]
dataset = dataset.loc[~dataset['to_url'].str.contains('/'),:]

dataset = dataset.loc[~dataset['from_url'].str.contains('\\\\'),:]
dataset = dataset.loc[~dataset['to_url'].str.contains('\\\\'),:]

dataset = dataset.loc[~dataset['from_url'].str.contains(':'),:]
dataset = dataset.loc[~dataset['to_url'].str.contains(':'),:]

# #Replacing ads. from urls
# dataset['from_url'] = dataset['from_url'].str.replace('.*ads.', '')
# dataset['to_url'] = dataset['to_url'].str.replace('.*ads.', '')

#Replacing www. from urls
dataset['from_url'] = dataset['from_url'].str.replace('.*www.', '')
dataset['to_url'] = dataset['to_url'].str.replace('.*www.', '')
dataset['from_url'] = dataset['from_url'].str.replace('.*www2.', '')
dataset['to_url'] = dataset['to_url'].str.replace('.*www2.', '')

#Filtering out the apparent explicit websites
searchfor = ['xxx', 'xvid', 'porn', 'xtube', 'sex']
dataset = dataset.loc[~dataset['from_url'].str.contains('|'.join(searchfor)), :]
dataset = dataset.loc[~dataset['to_url'].str.contains('|'.join(searchfor)), :]

dataset['from_url'] = dataset['from_url'].str.replace('.com', '.com|').str.split('|').str[0].str.strip()
dataset['to_url'] = dataset['to_url'].str.replace('.com', '.com|').str.split('|').str[0].str.strip()
dataset['from_url'] = dataset['from_url'].str.replace('.net', '.net|').str.split('|').str[0].str.strip()
dataset['to_url'] = dataset['to_url'].str.replace('.net', '.net|').str.split('|').str[0].str.strip()

# Keeping only the primary domain name
dataset.loc[dataset['from_url'].str.contains('.com'), 'from_url'] = dataset.loc[dataset['from_url'].str.contains('.com'), 'from_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['from_url'].str.contains('.net'), 'from_url'] = dataset.loc[dataset['from_url'].str.contains('.net'), 'from_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['from_url'].str.contains('.gov'), 'from_url'] = dataset.loc[dataset['from_url'].str.contains('.gov'), 'from_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['from_url'].str.contains('.edu'), 'from_url'] = dataset.loc[dataset['from_url'].str.contains('.edu'), 'from_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['from_url'].str.contains('.org'), 'from_url'] = dataset.loc[dataset['from_url'].str.contains('.org'), 'from_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))

dataset.loc[dataset['to_url'].str.contains('.com'), 'to_url'] = dataset.loc[dataset['to_url'].str.contains('.com'), 'to_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['to_url'].str.contains('.net'), 'to_url'] = dataset.loc[dataset['to_url'].str.contains('.net'), 'to_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['to_url'].str.contains('.gov'), 'to_url'] = dataset.loc[dataset['to_url'].str.contains('.gov'), 'to_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['to_url'].str.contains('.edu'), 'to_url'] = dataset.loc[dataset['to_url'].str.contains('.edu'), 'to_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))
dataset.loc[dataset['to_url'].str.contains('.org'), 'to_url'] = dataset.loc[dataset['to_url'].str.contains('.org'), 'to_url'].apply(lambda x: str(x.split('.')[-2])+'.'+str(x.split('.')[-1]))

# Filter out all urls with more than just the domain name
dataset['from_domain_sections'] = dataset['from_url'].apply(lambda x: len(x.split('.')))
dataset['to_domain_sections'] = dataset['to_url'].apply(lambda x: len(x.split('.')))

dataset = dataset.loc[(dataset['from_domain_sections']==2) & (dataset['to_domain_sections']==2), :]

dataset = dataset[['from_url', 'to_url', 'user_count']]

In [13]:
filtered_dataset = pd.DataFrame(dataset.groupby(['from_url', 'to_url'])['user_count'].sum()).reset_index()

In [14]:
# Filtering out the anomalous websites
filtered_dataset = filtered_dataset.loc[filtered_dataset['from_url'].apply(lambda x: len(x))>5,:]
filtered_dataset = filtered_dataset.loc[filtered_dataset['to_url'].apply(lambda x: len(x))>5,:]

filtered_dataset = filtered_dataset.loc[filtered_dataset['from_url'].apply(lambda x: len(x))<=25,:]
filtered_dataset = filtered_dataset.loc[filtered_dataset['to_url'].apply(lambda x: len(x))<=25,:]

In [15]:
# Filtering out all the entries that have the same to and from url
filtered_dataset = filtered_dataset.loc[filtered_dataset['to_url']!=filtered_dataset['from_url'], :]

In [16]:
filtered_dataset = filtered_dataset.loc[filtered_dataset['user_count']>50, :].sort_values('user_count', ascending = False)

Having performed the above steps for data cleaning & pre-processing, we're left with a dataset that has 3 columns with no anomalous values in them. This dataset is what will be used going forward for EDA and modelling (done in the next notebook file)

In [17]:
filtered_dataset.head()

,from_url,to_url,user_count
740565,youtube.com,google.com,212049
420365,indianapublicmedia.org,wfiu.org,188708
166391,drudgereport.com,google.com,57304
254742,google.com,boost.org,49318
252435,google.com,bio.net,43976


In [18]:
filtered_dataset.shape

(9729, 3)

In [19]:
filtered_dataset.to_csv('filtered_dataset.csv')